* Initialize the libraries and variables

In [ ]:
import { DOMParser } from "https://deno.land/x/deno_dom@v0.1.38/deno-dom-wasm.ts";
import { logError } from "./logError.js";
import { reloadEnv } from "./reloadEnv.js"
await reloadEnv();

const token = Deno.env.get("AEM_TOKEN");
const aemOrigin = 'https://author-p22655-e59433.adobeaemcloud.com';

* Fetch the cards to be proccessed
* Update the query

In [ ]:
const cardPath = '/content/dam/mas/sandbox';
const createdBy = '';  // with @adobe.com

const query = {
  filter: {
    path: cardPath,
    created: {
      by: [createdBy],
    }
  }  
};

const items = [];
let cursor = 'start';

while (cursor) {
  let url = `${aemOrigin}/adobe/sites/cf/fragments/search?query=${JSON.stringify(query)}`;
  
  if (cursor !== 'start') {
    url += `&cursor=${cursor}`;
  }

  const response = await fetch(url, {
    headers: {
      "x-api-key": "mas-studio",
      "Authorization": `Bearer ${token}`,
    } 
  });

  if (!response.ok) {
    logError(response);
  }

  const data = await response.json();
  
  items.push(...data.items);
  cursor = data.cursor;

  console.log(`Fetched ${data.items.length} items`);
}

console.log(`Total items: ${items.length}`);

* Review the items

In [ ]:
console.log(JSON.stringify(items[0], null, 2));

* Filter and keep cards to be modified
* Find cards that contain descriptions with links of the `secondary-link` class.

In [ ]:
const toBeModified = [];

for (const item of items) {
  const descHtml = item.fields.filter(f => f.name === 'description')[0].values[0];
  const doc = new DOMParser().parseFromString(descHtml, 'text/html');
  const links = doc.querySelectorAll('a');
  if ([...links].some(x => x.getAttribute('class')?.includes('secondary-link'))) {
    toBeModified.push(item);
  }
}

console.log(`Total items to be modified: ${toBeModified.length}`);

* Review cards to be updated

In [ ]:
console.log(JSON.stringify(toBeModified[0], null, 2));

* Updated the cards to be modified
* For each card, read the card first and keep etag and then update the card

In [ ]:
for (const item of toBeModified) {
  const cardId = item.id;
  const url = `${aemOrigin}/adobe/sites/cf/fragments/${cardId}`;

  console.log(`Getting card ${cardId}`);

  // Get the card
  const response = await fetch(url, {
    headers: {
      "x-api-key": "mas-studio",
      "Authorization": `Bearer ${token}`,
      pragma: 'no-cache',
      'cache-control': 'no-cache',
      'x-aem-affinity-type': 'api',
    }
  });

  if (!response.ok) {
    logError(response);
  }

  const data = await response.json();
  const etag = response.headers.get('etag');

  // Update the description

  // Get the current value
  const descField = data.fields.filter(f => f.name === 'description')[0];
  const descHtml = descField.values[0];

  console.log(`Before:\n ${descHtml}`);

  // Prepare the update
  const doc = new DOMParser().parseFromString(descHtml, 'text/html');
  const links = doc.querySelectorAll('a');
  for (const link of links) {
    if (link.getAttribute('class')?.includes('secondary-link')) {
      link.setAttribute('class', 'primary-link');
    }
  }
  descField.values[0] = doc.body.innerHTML;
  console.log(`Ready to update:\n ${descField.values[0]}`);

  // Update the card
  const responsePut = await fetch(url, {
    method: 'PUT',
    headers: {
      'Content-Type': 'application/json',
      "x-api-key": "mas-studio",
      "Authorization": `Bearer ${token}`,
      pragma: 'no-cache',
      'cache-control': 'no-cache',
      'x-aem-affinity-type': 'api',
      'If-Match': etag,
    },
    body: JSON.stringify({
      title: data.title,
      name: data.name,
      modelId: data.modelId,
      parentPath: data.parentPath,
      description: data.description,
      fields: data.fields,
    }),
  });

  if (!responsePut.ok) {
    logError(responsePut);
  }

  const dataPut = await responsePut.json();
  console.log(`After:\n ${dataPut.fields.filter(f => f.name === 'description')[0].values[0]}`);
}